<a href="https://colab.research.google.com/github/mattiabombieri/NeuralNetwork_Tesi_MB/blob/main/DataGenerationUnivariateNoise_MB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import math as mt
import matplotlib.pyplot as plt

2.18.0


In [2]:
a1 = 45 * np.sqrt(5)
a2 = 24 * np.sqrt(806115)
a3 = 1560 * np.sqrt(49407661)

def basis_fct(k, t):
    if k == 1:
        return 1 + 0 * t
    if k == 2:
        return -1 + np.exp(-t)
    if k == 3:
        return np.exp(-t) * t
    if k == 4:
        return (1/2) * (-2 * np.exp(-t) * t + np.exp(-t) * np.power(t, 2))
    if k == 5:
        return (-6 * np.exp(-t) * t - 36 * np.exp(-t) * np.power(t, 2) + np.exp(-t) * np.power(t, 3)) / a1
    if k == 6:
        return (-24 * np.exp(-t) * t - 192 * np.exp(-t) * np.power(t, 2) - 1440 * np.exp(-t) * np.power(t, 3) + np.exp(-t) * np.power(t, 4)) / a2
    if k == 7:
        return (-120 * np.exp(-t) * t - 1200 * np.exp(-t) * np.power(t, 2) - 10800 * np.exp(-t) * np.power(t, 3) + 100800 * np.exp(-t) * np.power(t, 4) + np.exp(-t) * np.power(t, 5)) / a3
    else:
        return 0

In [3]:
# Simulate one Brownian motionfrom 0 to T. We discretize daily
T = 1/12 # one month simulation time horizon
L = 30 # Discretization of BM
s = np.arange(0.0, 1/12, 1/(12*L))
Tminuss = T - s
print(Tminuss)
deltaS = 1 / (12*L)
Ssize = s.size

def evalCurve(x, BMInc, tau):
    M = x.size
    TauSize = tau.size
    Y0 = np.zeros(TauSize)
    drift = np.ones(TauSize)
    drift *= -(1/2) * T
    noise = np.ones(TauSize)
    noise *= mt.sqrt(T) * BMInc
    for k in range(0, M, 1):
        Y0 += x[k] * basis_fct(k + 1, tau + T)
        #tmp+=x[k]*np.power(t+T,k-1)*mt.exp(-(t+T))
    return np.exp(drift + noise + Y0)

[0.08333333 0.08055556 0.07777778 0.075      0.07222222 0.06944444
 0.06666667 0.06388889 0.06111111 0.05833333 0.05555556 0.05277778
 0.05       0.04722222 0.04444444 0.04166667 0.03888889 0.03611111
 0.03333333 0.03055556 0.02777778 0.025      0.02222222 0.01944444
 0.01666667 0.01388889 0.01111111 0.00833333 0.00555556 0.00277778]


In [4]:
import sys
print(sys.version_info)
!{sys.executable} -m pip install multiprocess

sys.version_info(major=3, minor=11, micro=11, releaselevel='final', serial=0)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.0 MB/s eta 0:00:00


In [5]:
from numpy.random import SeedSequence, default_rng
ss = SeedSequence(12345)

# Spawn off 10 child SeedSequences to pass to child processes.
child_seeds = ss.spawn(32)
streams = [default_rng(s) for s in child_seeds]

In [18]:
def magic_function(f):
    Strike = 1.0
    taugrid = np.arange(0.0, 1/12, 1/(12*L))
    TrainSizePerNode = 31250
    StartingValuesLocal = f[1].uniform(low = -0.5, high = 0.5, size = [TrainSizePerNode, 7])

    IncrementsBM = f[1].normal(0, 1, size = [TrainSizePerNode, 1])
    Y = np.zeros(TrainSizePerNode)

    for sample in range(0, TrainSizePerNode, 1):

        tmpvec = evalCurve(StartingValuesLocal[sample], IncrementsBM[sample], taugrid)
        tmp = np.sum(tmpvec) * (12/365)
        tmp = max(tmp - Strike, 0)
        Y[sample] = tmp
    return StartingValuesLocal, Y


def process_frame(f):
    return f[0], magic_function(f)

In [19]:
from tqdm import tqdm

from multiprocess import Pool
import time
tic = time.perf_counter()
#from magic_functions import process_frame

frames_list = [[i,streams[i]] for i in range(0,len(streams),1)]

max_pool = 32

with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.imap(process_frame,
                   frames_list),
            total=len(frames_list)
        )
    )

print(pool_outputs)
new_dict = dict(pool_outputs)
toc = time.perf_counter()
print(f"Calc time: {toc - tic:0.4f} seconds")

100%|██████████| 32/32 [03:56<00:00,  7.38s/it]  


[(0, (array([[ 0.36999885,  0.06088184, -0.27843507, ...,  0.05552605,
         0.27597337,  0.46055534],
       [ 0.41627211, -0.32638021,  0.30962175, ...,  0.10310086,
        -0.32621497, -0.0346609 ],
       [ 0.45123417,  0.28628765, -0.34419823, ...,  0.17506217,
        -0.22866537,  0.30101809],
       ...,
       [ 0.32942878,  0.09578549,  0.32353155, ..., -0.49993725,
        -0.43408409, -0.06353141],
       [ 0.11566463,  0.29291571,  0.32938953, ..., -0.3791963 ,
        -0.32774008,  0.19388667],
       [-0.32012208, -0.14495118,  0.22019503, ..., -0.04339812,
        -0.10938574, -0.45607476]]), array([0.70011372, 1.49083292, 0.        , ..., 0.        , 0.        ,
       0.        ]))), (1, (array([[-0.12292459,  0.18954752,  0.24347235, ..., -0.012945  ,
         0.18943038, -0.31699783],
       [ 0.30189439, -0.39355406, -0.06432063, ...,  0.486978  ,
        -0.31504875, -0.27410072],
       [-0.24313982, -0.1069087 , -0.48672306, ..., -0.31561184,
         0.0676

In [20]:
TrainSizePerNode=31250
TotalTrainSize=TrainSizePerNode*len(streams)
startingValues=np.zeros((TotalTrainSize,7))
Yvalues=np.zeros(TotalTrainSize)
for x in range(0, len(streams), 1):
    startingValues[x*TrainSizePerNode:(x+1)*TrainSizePerNode]=pool_outputs[x][1][0]
    Yvalues[x*TrainSizePerNode:(x+1)*TrainSizePerNode]=pool_outputs[x][1][1]
#print(startingValues)
print(Yvalues[3])
print(Yvalues.ndim)
np.save('startingvaluesUniVariate', startingValues)
np.save('outputsUniVariate',Yvalues)

0.02772772033287363
1
